# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [10]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [2]:
data = pd.read_csv("homepage_actions.csv")
data

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view
...,...,...,...,...
8183,2017-01-18 09:11:41.984113,192060,experiment,view
8184,2017-01-18 09:42:12.844575,755912,experiment,view
8185,2017-01-18 10:01:09.026482,458115,experiment,view
8186,2017-01-18 10:08:51.588469,505451,control,view


In [3]:
# Count the number of unique 'id' values for each action type ('view' and 'click')
viewers = set(data[data['action'] == 'view']['id'].unique())
clickers = set(data[data['action'] == 'click']['id'].unique())

# Find the intersection of viewers and clickers to determine how many viewers also clicked
viewers_clicked = len(viewers.intersection(clickers))

viewers_clicked

1860

In [5]:
clicks = data[data['action'] == 'click']
unique_click_ids = set(clicks['id'].unique())
view_without_click = data[(data['action'] == 'view') & (~data['id'].isin(unique_click_ids))]

if view_without_click.empty:
    print("There are no anomalies. Everyone who clicked also viewed.")
else:
    print("Anomalies found. Some users clicked without viewing.")
    print(view_without_click)

Anomalies found. Some users clicked without viewing.
                       timestamp      id       group action
0     2016-09-24 17:42:27.839496  804196  experiment   view
1     2016-09-24 19:19:03.542569  434745  experiment   view
2     2016-09-24 19:36:00.944135  507599  experiment   view
3     2016-09-24 19:59:02.646620  671993     control   view
4     2016-09-24 20:26:14.466886  536734  experiment   view
...                          ...     ...         ...    ...
8183  2017-01-18 09:11:41.984113  192060  experiment   view
8184  2017-01-18 09:42:12.844575  755912  experiment   view
8185  2017-01-18 10:01:09.026482  458115  experiment   view
8186  2017-01-18 10:08:51.588469  505451     control   view
8187  2017-01-18 10:24:08.629327  461199     control   view

[4468 rows x 4 columns]


In [27]:
control_group = data[data['group'] == 'control']
experiment_group = data[data['group'] == 'experiment']

# Get unique 'id' values for each group
unique_control_ids = set(control_group['id'].unique())
unique_experiment_ids = set(experiment_group['id'].unique())

# Find the intersection of 'id' values between the two groups
overlap_ids = unique_control_ids.intersection(unique_experiment_ids)

if len(overlap_ids) > 0:
    print("There is overlap between the control and experiment groups.")
    print(f"Overlap 'id' values: {overlap_ids}")
else:
    print("There is no overlap between the control and experiment groups.")

There is no overlap between the control and experiment groups.


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [33]:
data['count'] = 1
data.head(30)

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1
5,2016-09-24 20:32:25.712659,681598,experiment,view,1
6,2016-09-24 20:39:03.248853,522116,experiment,view,1
7,2016-09-24 20:57:20.336757,349125,experiment,view,1
8,2016-09-24 20:58:01.948663,349125,experiment,click,1
9,2016-09-24 21:00:12.278374,560027,control,view,1


In [36]:
control = data[data['group'] == 'control'].pivot_table(index='id', columns='action', values='count', fill_value=0)
experiment = data[data['group'] == 'experiment'].pivot_table(index='id', columns='action', values='count', fill_value=0)

print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control['click'].sum(), experiment['click'].sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control['click'].mean(), experiment['click'].mean()))
control.head()


Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932	Experiment: 928
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1,1
183089,0,1
183248,1,1
183515,0,1
183524,0,1


In [37]:
control_clicks = control['click']
experiment_clicks = experiment['click']

# Perform Welch's t-test
t_statistic, p_value = stats.ttest_ind(control_clicks, experiment_clicks, equal_var=False)

# Set significance level (alpha)
alpha = 0.05

# Check the p-value against the significance level
if p_value < alpha:
    print("The experimental homepage is statistically more effective than the control group's homepage.")
else:
    print("There is no statistically significant difference between the experimental and control homepages.")

# Print the t-statistic and p-value for reference
print(f"t-statistic: {t_statistic}")
print(f"p-value: {p_value}")

The experimental homepage is statistically more effective than the control group's homepage.
t-statistic: -2.615440020788211
p-value: 0.008932805628674201


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [39]:
control_ctr = control['click'].mean()

# Calculate the number of views in the experiment group
experiment_views = experiment['view'].sum()

# Calculate the expected number of clicks for the experiment group
expected_clicks_experiment = control_ctr * experiment_views

expected_clicks_experiment

838.0168067226891

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [49]:
x = len(experiment)
y = control_ctr
var = n * p * (1-p)
std = np.sqrt(var)
print(std)

24.568547907005815


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-scorecontrol_ctr.

In [51]:
cumulative_prob = stats.norm.cdf(num_std_deviations)

# Calculate the p-value (assuming a two-tailed test)
p_value = 2 * (1 - cumulative_prob)  # For a two-tailed test

p_value

0.0

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **The p-value calculated using the z-score and the normal distribution, based on the deviation of the actual number of clicks from the expected number of clicks does roughly match result of the previous statistical test**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.